# 🧠 Dendritic YOLOv8: PerforatedAI Hackathon

**Clean, reproducible notebook** for the PyTorch Dendritic Optimization Hackathon.

## What This Demonstrates
- Apply PerforatedAI dendritic optimization to YOLOv8n
- Compare baseline vs optimized model (params, speed, mAP)
- Edge deployment benefits: smaller, faster models

## Prerequisites
1. **Google Colab with GPU**: Runtime → Change runtime type → T4 GPU
2. **Run cells in order** from top to bottom
3. **Fresh runtime recommended**: Runtime → Restart runtime before starting

---
## Cell 1: Environment Setup (CRITICAL)

**Pin PyTorch < 2.6** to avoid `weights_only` unpickling errors.

In [ ]:
# === CLEAN ENVIRONMENT SETUP ===
# Pin PyTorch < 2.6 to avoid weights_only=True errors with YOLO checkpoints

!pip -q uninstall -y ultralytics wandb perforatedai || true

# Install known-good versions (PyTorch 2.4.1 works well with Colab GPU)
!pip -q install "torch==2.4.1" "torchvision==0.19.1" "torchaudio==2.4.1"
!pip -q install "ultralytics==8.2.0" "wandb" "matplotlib" "pandas" "seaborn"
!pip -q install "perforatedai==3.0.7"

print("✅ Dependencies installed!")

# Verify versions
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# === DEVICE SETUP ===
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🎯 Using device: {device}")

if device == 'cpu':
    print("⚠️ WARNING: Running on CPU. For best results:")
    print("   Runtime → Change runtime type → T4 GPU")
    print("   Then restart runtime and re-run from Cell 1")

In [ ]:
# === SANITY CHECK: YOLO loads without errors ===
from ultralytics import YOLO

print("Loading YOLOv8n...")
test_model = YOLO("yolov8n.pt")
print("✅ YOLO model loads successfully!")

# Quick validation to confirm everything works
print("\nRunning quick validation on COCO128...")
metrics = test_model.val(data="coco128.yaml", imgsz=640, device=device, verbose=False)
print(f"✅ Validation works! mAP50-95: {metrics.box.map:.4f}")

del test_model  # Clean up

---
## Cell 2: Imports & W&B Setup

In [ ]:
# === IMPORTS ===
import os
import time
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wandb
from ultralytics import YOLO

# PerforatedAI imports
try:
    from perforatedai import globals_perforatedai as GPA
    from perforatedai import utils_perforatedai as UPA
    PERFORATED_AI_AVAILABLE = True
    print("✅ PerforatedAI imported successfully!")
except ImportError as e:
    PERFORATED_AI_AVAILABLE = False
    print(f"⚠️ PerforatedAI not available: {e}")

print(f"\n📦 PerforatedAI: {'Available' if PERFORATED_AI_AVAILABLE else 'Not Available'}")

In [ ]:
# === W&B LOGIN ===
# Option 1: Set environment variable WANDB_API_KEY before running
# Option 2: Enter API key when prompted
# Option 3: Run offline (set WANDB_MODE=offline)

try:
    wandb.login()
    print("✅ W&B authenticated!")
except Exception as e:
    print(f"⚠️ W&B login failed: {e}")
    print("Running in offline mode...")
    os.environ["WANDB_MODE"] = "offline"

---
## Cell 3: Helper Functions

In [ ]:
# === HELPER FUNCTIONS ===

def count_parameters(model):
    """Count total and trainable parameters."""
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total, trainable

def measure_inference_speed(model, img_size=640, num_runs=50, device='cuda'):
    """Measure average inference time in milliseconds."""
    model.eval()
    dummy_input = torch.randn(1, 3, img_size, img_size).to(device)
    
    # Warmup
    for _ in range(10):
        with torch.no_grad():
            _ = model(dummy_input)
    
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    
    start = time.perf_counter()
    for _ in range(num_runs):
        with torch.no_grad():
            _ = model(dummy_input)
    
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    
    end = time.perf_counter()
    return (end - start) / num_runs * 1000

print("✅ Helper functions defined!")

---
## Section A: Baseline Training

In [ ]:
# === BASELINE: Load Model ===
print("🚀 Loading YOLOv8n baseline model...")

baseline_model = YOLO("yolov8n.pt")
baseline_model.model = baseline_model.model.to(device)

# Count parameters
baseline_params, _ = count_parameters(baseline_model.model)
print(f"📊 Baseline Parameters: {baseline_params / 1e6:.3f}M")

In [ ]:
# === BASELINE: Train ===
print("🏋️ Training baseline model (5 epochs)...")

wandb.init(
    project="Dendritic-YOLOv8-Hackathon",
    name="baseline-yolov8n",
    tags=["baseline", "yolov8n", "coco128"]
)

baseline_results = baseline_model.train(
    data="coco128.yaml",
    epochs=5,
    imgsz=640,
    batch=16,
    device=device,
    project="runs/baseline",
    name="yolov8n_baseline",
    exist_ok=True,
    verbose=True
)

print("✅ Baseline training complete!")

In [ ]:
# === BASELINE: Validate & Collect Metrics ===
print("📊 Validating baseline model...")

baseline_val = baseline_model.val(data="coco128.yaml", device=device)

baseline_metrics = {
    "params_M": baseline_params / 1e6,
    "mAP50": float(baseline_val.box.map50),
    "mAP50-95": float(baseline_val.box.map),
    "precision": float(baseline_val.box.mp),
    "recall": float(baseline_val.box.mr),
    "inference_ms": measure_inference_speed(baseline_model.model, device=device)
}

print("\n📊 BASELINE METRICS:")
for k, v in baseline_metrics.items():
    print(f"   {k}: {v:.4f}")

wandb.log({f"baseline_{k}": v for k, v in baseline_metrics.items()})
wandb.finish()

---
## Section B: Dendritic Optimization

In [ ]:
# === DENDRITIC: Load Fresh Model ===
print("🧠 Loading fresh YOLOv8n for dendritic optimization...")

dendritic_yolo = YOLO("yolov8n.pt")
dendritic_model = dendritic_yolo.model.to(device)

print(f"📊 Pre-optimization Parameters: {count_parameters(dendritic_model)[0] / 1e6:.3f}M")

In [ ]:
# === DENDRITIC: Apply PerforatedAI Optimization ===
print("🧠 Applying dendritic optimization...")

if PERFORATED_AI_AVAILABLE:
    # Configure PerforatedAI
    GPA.pc.set_testing_dendrite_capacity(False)
    GPA.pc.set_verbose(True)
    GPA.pc.set_dendrite_update_mode(True)
    
    # Save input stem (model[0]) before optimization to avoid weight mismatch
    input_stem = dendritic_model.model[0]
    
    # Apply dendritic optimization
    dendritic_model = UPA.initialize_pai(
        dendritic_model,
        doing_pai=True,
        save_name="DendriticYOLOv8",
        maximizing_score=True
    )
    
    # Restore input stem
    dendritic_model.model[0] = input_stem
    dendritic_model = dendritic_model.to(device)
    
    print("✅ Dendritic optimization applied!")
else:
    print("⚠️ PerforatedAI not available - using standard model")

dendritic_params, _ = count_parameters(dendritic_model)
print(f"📊 Post-optimization Parameters: {dendritic_params / 1e6:.3f}M")

In [ ]:
# === DENDRITIC: Train ===
print("🏋️ Training dendritic model (5 epochs)...")

wandb.init(
    project="Dendritic-YOLOv8-Hackathon",
    name="dendritic-yolov8n",
    tags=["dendritic", "perforatedai", "yolov8n", "coco128"]
)

# Reassign modified model to YOLO wrapper
dendritic_yolo.model = dendritic_model

dendritic_results = dendritic_yolo.train(
    data="coco128.yaml",
    epochs=5,
    imgsz=640,
    batch=16,
    device=device,
    project="runs/dendritic",
    name="yolov8n_dendritic",
    exist_ok=True,
    verbose=True,
    optimizer="Adam",
    lr0=0.001
)

print("✅ Dendritic training complete!")

In [ ]:
# === DENDRITIC: Validate & Collect Metrics ===
print("📊 Validating dendritic model...")

dendritic_val = dendritic_yolo.val(data="coco128.yaml", device=device)

dendritic_metrics = {
    "params_M": dendritic_params / 1e6,
    "mAP50": float(dendritic_val.box.map50),
    "mAP50-95": float(dendritic_val.box.map),
    "precision": float(dendritic_val.box.mp),
    "recall": float(dendritic_val.box.mr),
    "inference_ms": measure_inference_speed(dendritic_yolo.model, device=device)
}

print("\n📊 DENDRITIC METRICS:")
for k, v in dendritic_metrics.items():
    print(f"   {k}: {v:.4f}")

wandb.log({f"dendritic_{k}": v for k, v in dendritic_metrics.items()})
wandb.finish()

---
## Section C: Comparison & Results

In [ ]:
# === COMPARISON ===
print("\n" + "="*70)
print("📊 BASELINE vs DENDRITIC COMPARISON")
print("="*70)

# Calculate deltas
param_reduction = ((baseline_metrics['params_M'] - dendritic_metrics['params_M']) / baseline_metrics['params_M']) * 100
map50_change = dendritic_metrics['mAP50'] - baseline_metrics['mAP50']
map_change = dendritic_metrics['mAP50-95'] - baseline_metrics['mAP50-95']
speed_improvement = ((baseline_metrics['inference_ms'] - dendritic_metrics['inference_ms']) / baseline_metrics['inference_ms']) * 100

print(f"\n{'Metric':<20} {'Baseline':>12} {'Dendritic':>12} {'Delta':>12}")
print("-" * 60)
print(f"{'Parameters (M)':<20} {baseline_metrics['params_M']:>12.3f} {dendritic_metrics['params_M']:>12.3f} {param_reduction:>+11.1f}%")
print(f"{'mAP50':<20} {baseline_metrics['mAP50']:>12.4f} {dendritic_metrics['mAP50']:>12.4f} {map50_change:>+11.4f}")
print(f"{'mAP50-95':<20} {baseline_metrics['mAP50-95']:>12.4f} {dendritic_metrics['mAP50-95']:>12.4f} {map_change:>+11.4f}")
print(f"{'Inference (ms)':<20} {baseline_metrics['inference_ms']:>12.2f} {dendritic_metrics['inference_ms']:>12.2f} {speed_improvement:>+11.1f}%")
print("="*70)

print(f"\n🏆 KEY RESULTS:")
print(f"   📦 Parameter Reduction: {param_reduction:+.1f}%")
print(f"   ⚡ Speed Improvement: {speed_improvement:+.1f}%")
print(f"   🎯 mAP50 Change: {map50_change:+.4f}")

In [ ]:
# === VISUALIZATION ===
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Chart 1: mAP Comparison
metrics_names = ['mAP50', 'mAP50-95']
x = np.arange(len(metrics_names))
width = 0.35

axes[0].bar(x - width/2, [baseline_metrics['mAP50'], baseline_metrics['mAP50-95']], width, label='Baseline', color='steelblue')
axes[0].bar(x + width/2, [dendritic_metrics['mAP50'], dendritic_metrics['mAP50-95']], width, label='Dendritic', color='coral')
axes[0].set_ylabel('Score')
axes[0].set_title('Accuracy Comparison')
axes[0].set_xticks(x)
axes[0].set_xticklabels(metrics_names)
axes[0].legend()

# Chart 2: Parameters
axes[1].bar(['Baseline', 'Dendritic'], [baseline_metrics['params_M'], dendritic_metrics['params_M']], color=['steelblue', 'coral'])
axes[1].set_ylabel('Parameters (Millions)')
axes[1].set_title('Model Size')
for i, v in enumerate([baseline_metrics['params_M'], dendritic_metrics['params_M']]):
    axes[1].text(i, v + 0.05, f'{v:.2f}M', ha='center')

# Chart 3: Inference Speed
axes[2].bar(['Baseline', 'Dendritic'], [baseline_metrics['inference_ms'], dendritic_metrics['inference_ms']], color=['steelblue', 'coral'])
axes[2].set_ylabel('Inference Time (ms)')
axes[2].set_title('Speed Comparison')
for i, v in enumerate([baseline_metrics['inference_ms'], dendritic_metrics['inference_ms']]):
    axes[2].text(i, v + 0.5, f'{v:.1f}ms', ha='center')

plt.tight_layout()
plt.savefig('comparison_chart.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Chart saved to 'comparison_chart.png'")

In [ ]:
# === SAVE RESULTS ===
results = {
    "hackathon": "PyTorch Dendritic Optimization Hackathon",
    "model": "YOLOv8n",
    "dataset": "COCO128",
    "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    "baseline": baseline_metrics,
    "dendritic": dendritic_metrics,
    "improvements": {
        "parameter_reduction_pct": param_reduction,
        "speed_improvement_pct": speed_improvement,
        "mAP50_change": map50_change,
        "mAP50-95_change": map_change
    }
}

with open('hackathon_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("✅ Results saved to 'hackathon_results.json'")
print("\n📋 SUBMISSION READY!")
print("   1. Download comparison_chart.png and hackathon_results.json")
print("   2. Include in your PR to PerforatedAI/PerforatedAI")
print("   3. Submit PR link on Devpost")